## Records

Dealing with entities that do not belong to a well-defined group of cases.

We previously presented the need to add useful information from some entities that cannot be restricted to having specific types, for example, credit card data.

Virtually any high-level language provides a way to define such types. In C one has the type `struct`, in Python a class with no methods (or an [named tuple](https://docs.python.org/3/library/collections.html#collections.namedtuple)), etc. This type in F\# is called _record_.

In [1]:
type CreditCard =
    {
        HoldersName : string
        Number: string
        ExpirationDateMonth: uint8 
        ExpirationDateYear: uint8 
        CVV: uint16
    }

The record type uses tags to aggregate the different components of the type. Each component has a tag (`HoldersName`, `Number`, etc.) and a type associated with it.
To create a record type, it is necessary to define each and every component:

In [2]:
let doeCard = 
    {
        HoldersName = "John Doe"
        Number = "1234 5678 9101 1121"
        ExpirationDateMonth = 12uy
        ExpirationDateYear = 23uy 
        CVV = 111us 
    }


> 🔔 Note the suffix `uy` for 8-bit unsigned integers (`uint8`) and `us` for its 16-bit companion (`uint16`).

Instead of indenting the definition of the value `doeCard` across different lines, one can write all the components together, separated by `;`:

In [3]:
let doeFakeCard = { HoldersName = "John Doe"; Number = "1234 5678 9101 1121"; ExpirationDateMonth = 12uy; ExpirationDateYear = 23uy ; CVV = 111us}

printfn "%A" doeFakeCard    

{ HoldersName = "John Doe"
  Number = "1234 5678 9101 1121"
  ExpirationDateMonth = 12uy
  ExpirationDateYear = 23uy
  CVV = 111us }


but as you can see, writing a one-liner to define a record value is adequate [only for small records](https://learn.microsoft.com/en-us/dotnet/fsharp/style-guide/formatting#formatting-record-expressions).


What happens if the John Doe card expires and needs to be replaced with a new one?
As with all other values ​​in the language, records are _immutable_, so it is not possible to update `doeCard` instead. To do that, we need to create another new value. F\# provides a way to _copy and update_ a record value, which allows us to change only the components that need to be changed in a record. Assuming the new card keeps the number (and of course the cardholder's name), we would have:

In [4]:
let newDoeCard = 
    { doeCard with 
        ExpirationDateMonth = 12uy
        ExpirationDateYear = 25uy
        CVV = 222us 
    }

printfn "%A" newDoeCard    

{ HoldersName = "John Doe"
  Number = "1234 5678 9101 1121"
  ExpirationDateMonth = 12uy
  ExpirationDateYear = 25uy
  CVV = 222us }


One again uses tags to express the type of record. Remember that values are inmutable, therefore, this creates a _new_ record with the old `doeCard` value, using `with` to identify the components that need to be updated.

To access a specific component of a record, use `.` again, as we did with discriminated unions:

In [5]:
printfn "John's Does card number: %A" newDoeCard.Number 
printfn "John's Does card CVV: %A" newDoeCard.CVV

John's Does card number: "1234 5678 9101 1121"
John's Does card CVV: 222us


## Mixing types

The discriminated union and the record are the two ways one can represent entities in the language. One can build all kinds of complex (;-D) types by mixing them, it is up to the programmer how to combine these smaller bricks to model the domain.

For example, one can wrap the expiration date into its own type:

In [6]:
type ExpirationDate =
    { 
        Month : uint8
        Year : uint8 
    }

That would give us a cleaner `CreditCard2` type:

In [7]:
type CreditCard2 =
    {
        HoldersName : string
        Number: string
        ExpirationDate: ExpirationDate 
        CVV: uint16
    }

For the vending machine, you can write

In [8]:
type FoodProduct =
    | Chips
    | Chocolate
    | Candy 

type BrandedFood =
    | Chips of string 
    | Chocolate of string 
    | Candy of string     

type FoodMachineItem =
    {
        Brand: BrandedFood
        ProductType: FoodProduct 
        Price: float 
    }

In [9]:
let sourCandy = {
    Brand = BrandedFood.Candy "TearDrops"
    ProductType = FoodProduct.Candy 
    Price = 2.39
}

Note that we need to fully specify the type in the `ProductType` component, using `FoodProduct.Candy`. This is to avoid collision with the `Candy of string` case on the `BrandedFood` type. Don't worry! The compiler behind will cover it, pointing out the problem:

In [10]:
let sourCandyWithCollision = {
    Brand = BrandedFood.Candy "TearDrops"
    ProductType = Candy 
    Price = 2.39
}

Error: input.fsx (3,19)-(3,24) typecheck error This expression was expected to have type
    'FoodProduct'    
but here has type
    'string -> BrandedFood'    